# Random Forests for waiting times prediction

In [20]:
import pandas as pd
import numpy as np
import matplotlib as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.metrics import classification_report
from tqdm import tqdm
from statistics import mean
import math
from statistics import mean
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score, accuracy_score
from timeit import default_timer as timer
from datetime import timedelta

from sklearn.ensemble import RandomForestClassifier

In [5]:
hp_oHe = pd.read_csv('HP_OHE_3class.csv')
hp_oHe.drop(hp_oHe.tail(17).index,inplace=True) 
hp_oHe = hp_oHe.drop('Unnamed: 0',axis=1)

hp_ME = pd.read_csv("harryPotterClean.csv")
hp_ME.drop(hp_ME.tail(17).index,inplace=True) 
hp_ME = hp_ME.drop('Unnamed: 0',axis=1)

hp_OE = pd.read_csv("harryPotterCleanOE.csv")
hp_OE.drop(hp_OE.tail(17).index,inplace=True) 
hp_OE = hp_OE.drop('Unnamed: 0',axis=1)

In [6]:
def getXandY(df):
    df.drop(df.tail(20).index,inplace=True) 
    x = df.drop(['HP_Forbidden_clean'],axis=1)
    y = df.HP_Forbidden_clean
    return(x,y)

def trainTest(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True)
    return(X_train, X_test, y_train, y_test)

#Function to perform oversampling
def overSampling(X_train, y_train, y_test, method):
    X_train_os, y_train_os= method.fit_resample(X_train, y_train)
    # Check the number of records after over sampling
    #print(sorted(Counter(y_train_os).items())) 
    return(X_train_os, y_train_os)

smote = SMOTE(random_state=42)


def testModel(df,var_order,n_vars,n_loops,method):
    highest = 0
    for j in tqdm(range(1,n_vars)):
        #split our dataframe into X and Y
        x,y=getXandY(df)
        #create the lists to store metrics
        acc = []
        rec = []
        preci = []
        f1 = []
        for i in range(n_loops):
            #split the dataFrame into test and train
            X_train, X_test, y_train, y_test = trainTest(x,y)
            #Oversample the train dataset with SMOTE
            X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
            #define the variables order 
            X_train_os_r = X_train_os[var_order]
            X_test_r = X_test[var_order]
            df1= X_train_os_r.iloc[:, 0:j] #use only part of the variables
            
            #create and train decision trees
            rnd_clf = RandomForestClassifier(n_jobs=-1)
            rnd_clf.fit(df1, y_train_os)
        
            y_pred=rnd_clf.predict(X_test_r.iloc[:, 0:j])
            ac=metrics.accuracy_score(y_test, y_pred)
            acc.append(ac)
            p=metrics.precision_score(y_test, y_pred,average='macro')
            preci.append(p)
            r=metrics.recall_score(y_test, y_pred,average='macro')
            rec.append(r)
            f=metrics.f1_score(y_test, y_pred, average='macro')
            f1.append(f)
        print(df1.columns)
        print("For {} features: \n Accuracy: {} \n Precision: {} \n Recall: {} \n F1 score: {}".format(
        j,mean(acc),mean(preci),mean(rec),mean(f1)))
        
        if mean(acc)>highest:
            highest = mean(acc)
            best = "best accuracy = {}, with {} features, with {}".format(mean(acc),j,method)
        print(best)
        #print(classification_report(y_test, y_pred))
    print(best)
        
def analizeDF(df,order,n_vars,n_loops):
    for i in range(len(order)):
        print('------------------------- Analyzing method {} -------------------------'.format(method[i]))
        print('The variable order is: \n {}'.format(order[i]))
        testModel(df,order[i],n_vars,n_loops,method[i])
        print('\n \n')

## One hot encoding

In [4]:
#Variable order in one hot encoding:
pear_corrO = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'fog',	'overcast clouds',	'heavy intensity rain',	'minute',	'haze',	'thunderstorm with light rain',	'scattered clouds',	'clear sky',	'mist',	'light intensity drizzle',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'year',	'light rain',	'humidity',	'hour',	'Pandemic']
kend_corrO = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'heavy intensity rain',	'fog',	'year',	'overcast clouds',	'minute',	'haze',	'thunderstorm with light rain',	'mist',	'light intensity drizzle',	'scattered clouds',	'clear sky',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'light rain',	'humidity',	'hour',	'Pandemic']
mutInf_classO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_regO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_class2O = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek',	'year',	'holiday',	'shower rain',	'light rain',	'thunderstorm',	'fog',	'broken clouds',	'Pandemic',	'thunderstorm with rain',	'light intensity drizzle',	'thunderstorm with light rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds',	'clear sky']
varThreO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',]
mrmrO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds']

orderOHE=[pear_corrO,kend_corrO,mutInf_classO,mutInf_regO,mutInf_class2O,varThreO,mrmrO]

method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'varianceThreshold','MRMR']

In [7]:
analizeDF(hp_oHe,orderOHE,28,n_loops=20)

  0%|          | 0/27 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'shower rain', 'broken clouds', 'fog', 'overcast clouds', 'heavy intensity rain', 'minute', 'haze', 'thunderstorm with light rain', 'scattered clouds', 'clear sky', 'mist', 'light intensity drizzle', 'few clouds', 'thunderstorm', 'very heavy rain', 'moderate rain', 'thunderstorm with rain', 'year', 'light rain', 'humidity', 'hour', 'Pandemic']


  4%|▎         | 1/27 [00:16<07:07, 16.43s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.4388303708580206 
 Precision: 0.3640619531022423 
 Recall: 0.3630652794614665 
 F1 score: 0.362626551943934
best accuracy = 0.4388303708580206, with 1 features, with Pearson_correlation


  7%|▋         | 2/27 [00:34<07:12, 17.29s/it]

...

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute'],
      dtype='object')
For 25 features: 
 Accuracy: 0.875268817204301 
 Precision: 0.8423664161203477 
 Recall: 0.8408805918783768 
 F1 score: 0.8415368393769003
best accuracy = 0.8867821067821068, with 21 features, with MRMR


 96%|█████████▋| 26/27 [04:31<00:10, 10.84s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze'],
      dtype='object')
For 26 features: 
 Accuracy: 0.8738573508005822 
 Precision: 0.8394613062315774 
 Recall: 0.8402784227770286 
 F1 score: 0.8397823000735265
best accuracy = 0.8867821067821068, with 21 features, with MRMR


100%|██████████| 27/27 [04:42<00:00, 10.46s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze', 'few clouds'],
      dtype='object')
For 27 features: 
 Accuracy: 0.8753864100320793 
 Precision: 0.8423370889282702 
 Recall: 0.8394051722804096 
 F1 score: 0.8407401236856014
best accuracy = 0.8867821067821068, with 21 features, with MRMR
best accuracy = 0.8867821067821068, with 21 features, with MRMR

 



## Manual Encoding

In [ ]:
#Replace times ending in 5 and also compacting the rest to have only 6 classes
b=hp_ME.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df3=pd.DataFrame(b)
df3.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis3=pd.concat([hp_ME, df3], axis=1)
hp3 = hp_bis3.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp3.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)
hp3 = hp3[hp3.HP_Forbidden_clean != 0] #delete rows with 0 min
hp3.HP_Forbidden_clean.unique()

In [ ]:
HP_ME = hp3

pear_corr = ['temperature','holiday','day','month','pressure','dayOfTheWeek',
            'report','minute','year','humidity','hour',	'Pandemic']
kend_corr = ['temperature','holiday','month','day','pressure','dayOfTheWeek','report','year',
             'minute','humidity','hour','Pandemic']
mutInf_class = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_reg = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_class2 = ['day','temperature','month','humidity','hour','dayOfTheWeek','pressure',
                 'holiday','year','report','minute','Pandemic']
varThre = ['month','day','year','hour','minute','holiday','dayOfTheWeek',
           'temperature','humidity','pressure','report']
mrmr = ['temperature','dayOfTheWeek','hour','holiday','humidity','day',
        'year','month','pressure','minute','report']
method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'variableThreshold','MRMR']

orderME=[pear_corr,kend_corr,mutInf_class,mutInf_reg,mutInf_class2,varThre,mrmr]

In [ ]:
analizeDF(HP_ME,orderME,13,n_loops=20)

## Ordinal Encoding

In [ ]:
hp_OE.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)

#Replace times ending in 5 and also compacting the rest to have only 6 classes
a=hp_OE.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df1=pd.DataFrame(a)
df1.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis=pd.concat([hp_OE, df1], axis=1)
hp2 = hp_bis.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp2 = hp2[hp2.HP_Forbidden_clean != 0] #delete rows with 0 min
hp2.HP_Forbidden_clean.unique()

#Variable order in ordinal encoding:
pear_corrOE = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'report',	'minute',	'year',	'humidity',	'hour',	'Pandemic']
kend_corrOE = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'report',	'year',	'minute',	'humidity',	'hour',	'Pandemic',]
mutInf_classOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_regOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_class2OE = ['day',	'temperature',	'month',	'humidity',	'hour',	'dayOfTheWeek',	'pressure',	'holiday',	'year',	'report',	'minute',	'Pandemic',]
varThreOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'report']
mrmrOE = ['temperature',	'dayOfTheWeek',	'hour',	'holiday',	'humidity',	'day',	'year',	'month',	'pressure',	'minute',	'report']

orderOE=[pear_corrOE,kend_corrOE,mutInf_classOE,mutInf_regOE,mutInf_class2OE,varThreOE,mrmrOE]

In [ ]:
analizeDF(hp2,orderOE,13,n_loops=20)

## Hyperparameter Tuning

In [17]:
bootstrap1= [True, False]
max_depth1=[10, 20, 50, 80, 100, None]
max_features1= ['auto', 'sqrt']
min_samples_leaf1= [1, 2, 4, 8, 10]
min_samples_split1= [2, 5, 10]
n_estimators1= [200, 500, 1000, 1500]

In [18]:
def tuneRandomF(X,Y,mu,bootstrap1,max_depth1,max_features1,min_samples_leaf1,
                min_samples_split1,n_estimators1,loops):
    arr = []
    highest=0
    for a in range(len(bootstrap1)):
        for b in range(len(max_depth1)):
            for c in range(len(max_features1)):
                for d in range(len(min_samples_leaf1)):
                    for e in range(len(min_samples_split1)):
                        for f in range(len(n_estimators1)):
                            start = timer()
                            for l in range(loops):
                                start = timer()
                                X_train, X_test, y_train, y_test = trainTest(X,Y)
                                #Oversample the train dataset with SMOTE
                                X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
                                #define the variables order 
                                X_train_os_r = X_train_os[mu]
                                X_test_r = X_test[mu]
                                rnd_clf = RandomForestClassifier(bootstrap=bootstrap1[a],
                                                                 max_depth=max_depth1[b],
                                                                 max_features=max_features1[c],
                                                                 min_samples_leaf=min_samples_leaf1[d],
                                                                 min_samples_split=min_samples_split1[e],
                                                                 n_estimators=n_estimators1[f])
                                rnd_clf.fit(X_train_os_r,y_train_os)
                                y_pred=rnd_clf.predict(X_test_r)
                                score = accuracy_score(y_test, rnd_clf.predict(X_test_r))
                                arr.append(score)
                            print("For the parameters boots:{}, mx_depth:{}, mx_fts:{} "
                                  "min_leaf:{}, min_splt:{}, n_stim {} accuracy is:".format(bootstrap1[a],
                                                                 max_depth1[b],
                                                                 max_features1[c],
                                                                 min_samples_leaf1[d],
                                                                 min_samples_split1[e],
                                                                 n_estimators1[f]))
                            mean_acc=mean(arr)
                            print(mean_acc)
                            end1 = timer()
                            print(timedelta(seconds=end1-start))
                            #check the best configuration
                            if mean_acc > highest:
                                highest = mean_acc
                                description1 = "---------------Best values = "
                                d12="boots:{}, mx_depth:{}, mx_fts:{}, min_leaf:{}, ".format(bootstrap1[a],
                                                                                         max_depth1[b],
                                                                                         max_features1[c],
                                                                                         min_samples_leaf1[d])
                                description2= "min_splt:{}, n_stim{} with accuracy: {}---------".format(
                                                                                         min_samples_split1[e],
                                                                                         n_estimators1[f],
                                                                                         mean_acc)
                            else:
                                highest = highest
                            arr = []
        print(description1+d12+description2)

In [23]:
X,Y=getXandY(hp_oHe)
mrmr = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	
        'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',
        'few clouds',	'thunderstorm',	'shower rain',	'heavy intensity rain',	'mist',	'scattered clouds']
tuneRandomF(X,Y,mrmr,bootstrap1,max_depth1,max_features1,min_samples_leaf1,min_samples_split1,n_estimators1,1)


For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:1, min_splt:2, n_stim 200 accuracy is:
0.7897666452579747
0:00:02.901910
For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:1, min_splt:2, n_stim 500 accuracy is:
0.775208734746307
0:00:06.964034
For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:1, min_splt:2, n_stim 1000 accuracy is:
0.7820595161635624
0:00:12.957144
For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:1, min_splt:2, n_stim 1500 accuracy is:
0.7822736030828517
0:00:18.432513
For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:1, min_splt:5, n_stim 200 accuracy is:
0.7839862984371655
0:00:02.862190
For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:1, min_splt:5, n_stim 500 accuracy is:
0.7745664739884393
0:00:06.350133
For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:1, min_splt:5, n_stim 1000 accuracy is:
0.7745664739884393
0:00:12.313840
For the parameters boots:True, m

For the parameters boots:True, mx_depth:10, mx_fts:auto min_leaf:10, min_splt:10, n_stim 1500 accuracy is:
0.7619353457503747
0:00:19.067577
For the parameters boots:True, mx_depth:10, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 200 accuracy is:
0.7820595161635624
0:00:03.423075
For the parameters boots:True, mx_depth:10, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 500 accuracy is:
0.7859130807107686
0:00:06.730940
For the parameters boots:True, mx_depth:10, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 1000 accuracy is:
0.7837722115178762
0:00:13.442186
For the parameters boots:True, mx_depth:10, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 1500 accuracy is:
0.7867694283879255
0:00:20.339853
For the parameters boots:True, mx_depth:10, mx_fts:sqrt min_leaf:1, min_splt:5, n_stim 200 accuracy is:
0.7794904731320916
0:00:03.165583
For the parameters boots:True, mx_depth:10, mx_fts:sqrt min_leaf:1, min_splt:5, n_stim 500 accuracy is:
0.7829158638407193
0:00:06.935242
For the parameters boots:True

For the parameters boots:True, mx_depth:10, mx_fts:sqrt min_leaf:10, min_splt:10, n_stim 1500 accuracy is:
0.7690002140869193
0:00:18.567594
For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:1, min_splt:2, n_stim 200 accuracy is:
0.8670520231213873
0:00:04.161564
For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:1, min_splt:2, n_stim 500 accuracy is:
0.8728323699421965
0:00:10.164949
For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:1, min_splt:2, n_stim 1000 accuracy is:
0.8638407193320488
0:00:20.019877
For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:1, min_splt:2, n_stim 1500 accuracy is:
0.8670520231213873
0:00:29.880251
For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:1, min_splt:5, n_stim 200 accuracy is:
0.8580603725112396
0:00:04.196615
For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:1, min_splt:5, n_stim 500 accuracy is:
0.8559195033183472
0:00:09.912964
For the parameters boots:True

For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:10, min_splt:10, n_stim 1000 accuracy is:
0.8248769000214087
0:00:16.528124
For the parameters boots:True, mx_depth:20, mx_fts:auto min_leaf:10, min_splt:10, n_stim 1500 accuracy is:
0.8066795118818241
0:00:24.661311
For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 200 accuracy is:
0.8653393277670735
0:00:04.318365
For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 500 accuracy is:
0.8730464568614857
0:00:10.121421
For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 1000 accuracy is:
0.8756154998929565
0:00:18.955875
For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:1, min_splt:2, n_stim 1500 accuracy is:
0.8636266324127596
0:00:29.378645
For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:1, min_splt:5, n_stim 200 accuracy is:
0.8582744594305288
0:00:04.395653
For the parameters boots:T

For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:10, min_splt:10, n_stim 500 accuracy is:
0.805823164204667
0:00:08.354155
For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:10, min_splt:10, n_stim 1000 accuracy is:
0.8188824662813102
0:00:16.471274
For the parameters boots:True, mx_depth:20, mx_fts:sqrt min_leaf:10, min_splt:10, n_stim 1500 accuracy is:
0.818668379362021
0:00:24.743959
For the parameters boots:True, mx_depth:50, mx_fts:auto min_leaf:1, min_splt:2, n_stim 200 accuracy is:
0.8764718475701134
0:00:04.336127
For the parameters boots:True, mx_depth:50, mx_fts:auto min_leaf:1, min_splt:2, n_stim 500 accuracy is:
0.8702633269107257
0:00:10.130268
For the parameters boots:True, mx_depth:50, mx_fts:auto min_leaf:1, min_splt:2, n_stim 1000 accuracy is:
0.8771141083279812
0:00:19.606136
For the parameters boots:True, mx_depth:50, mx_fts:auto min_leaf:1, min_splt:2, n_stim 1500 accuracy is:
0.8743309783772212
0:00:30.918378
For the parameters boots:T

...

For the parameters boots:False, mx_depth:80, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 200 accuracy is:
0.8349389852280026
0:00:04.867010
For the parameters boots:False, mx_depth:80, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 500 accuracy is:
0.8372939413401841
0:00:10.601233
For the parameters boots:False, mx_depth:80, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 1000 accuracy is:
0.8441447227574396
0:00:22.153634
For the parameters boots:False, mx_depth:80, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 1500 accuracy is:
0.8268036822950118
0:00:32.001247
For the parameters boots:False, mx_depth:80, mx_fts:sqrt min_leaf:8, min_splt:10, n_stim 200 accuracy is:
0.8362235067437379
0:00:04.621421
For the parameters boots:False, mx_depth:80, mx_fts:sqrt min_leaf:8, min_splt:10, n_stim 500 accuracy is:
0.827874116891458
0:00:10.731432
For the parameters boots:False, mx_depth:80, mx_fts:sqrt min_leaf:8, min_splt:10, n_stim 1000 accuracy is:
0.8364375936630272
0:00:21.254854
For the parameters boo

For the parameters boots:False, mx_depth:100, mx_fts:auto min_leaf:8, min_splt:2, n_stim 1500 accuracy is:
0.8220937700706487
0:00:34.788242
For the parameters boots:False, mx_depth:100, mx_fts:auto min_leaf:8, min_splt:5, n_stim 200 accuracy is:
0.8338685506315564
0:00:04.837055
For the parameters boots:False, mx_depth:100, mx_fts:auto min_leaf:8, min_splt:5, n_stim 500 accuracy is:
0.8272318561335903
0:00:11.599413
For the parameters boots:False, mx_depth:100, mx_fts:auto min_leaf:8, min_splt:5, n_stim 1000 accuracy is:
0.8415756797259687
0:00:23.227973
For the parameters boots:False, mx_depth:100, mx_fts:auto min_leaf:8, min_splt:5, n_stim 1500 accuracy is:
0.831727681438664
0:00:34.020255
For the parameters boots:False, mx_depth:100, mx_fts:auto min_leaf:8, min_splt:10, n_stim 200 accuracy is:
0.8272318561335903
0:00:04.774159
For the parameters boots:False, mx_depth:100, mx_fts:auto min_leaf:8, min_splt:10, n_stim 500 accuracy is:
0.8345108113894241
0:00:12.392738
For the paramete

For the parameters boots:False, mx_depth:100, mx_fts:sqrt min_leaf:8, min_splt:2, n_stim 1000 accuracy is:
0.8338685506315564
0:00:23.932060
For the parameters boots:False, mx_depth:100, mx_fts:sqrt min_leaf:8, min_splt:2, n_stim 1500 accuracy is:
0.8432883750802826
0:00:35.720692
For the parameters boots:False, mx_depth:100, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 200 accuracy is:
0.8413615928066795
0:00:04.750650
For the parameters boots:False, mx_depth:100, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 500 accuracy is:
0.8212374223934917
0:00:11.026150
For the parameters boots:False, mx_depth:100, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 1000 accuracy is:
0.8379362020980519
0:00:20.794928
For the parameters boots:False, mx_depth:100, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 1500 accuracy is:
0.8360094198244488
0:00:32.570051
For the parameters boots:False, mx_depth:100, mx_fts:sqrt min_leaf:8, min_splt:10, n_stim 200 accuracy is:
0.8302290730036395
0:00:04.666609
For the paramet

For the parameters boots:False, mx_depth:None, mx_fts:auto min_leaf:8, min_splt:2, n_stim 200 accuracy is:
0.8248769000214087
0:00:05.159715
For the parameters boots:False, mx_depth:None, mx_fts:auto min_leaf:8, min_splt:2, n_stim 500 accuracy is:
0.8366516805823164
0:00:11.871060
For the parameters boots:False, mx_depth:None, mx_fts:auto min_leaf:8, min_splt:2, n_stim 1000 accuracy is:
0.827017769214301
0:00:22.892476
For the parameters boots:False, mx_depth:None, mx_fts:auto min_leaf:8, min_splt:2, n_stim 1500 accuracy is:
0.8377221151787626
0:00:33.979698
For the parameters boots:False, mx_depth:None, mx_fts:auto min_leaf:8, min_splt:5, n_stim 200 accuracy is:
0.8345108113894241
0:00:05.055833
For the parameters boots:False, mx_depth:None, mx_fts:auto min_leaf:8, min_splt:5, n_stim 500 accuracy is:
0.8441447227574396
0:00:12.138649
For the parameters boots:False, mx_depth:None, mx_fts:auto min_leaf:8, min_splt:5, n_stim 1000 accuracy is:
0.8332262898736887
0:00:23.028011
For the par

For the parameters boots:False, mx_depth:None, mx_fts:sqrt min_leaf:4, min_splt:10, n_stim 1000 accuracy is:
0.8452151573538856
0:00:23.157484
For the parameters boots:False, mx_depth:None, mx_fts:sqrt min_leaf:4, min_splt:10, n_stim 1500 accuracy is:
0.8533504602868764
0:00:35.272397
For the parameters boots:False, mx_depth:None, mx_fts:sqrt min_leaf:8, min_splt:2, n_stim 200 accuracy is:
0.8287304645686149
0:00:04.623055
For the parameters boots:False, mx_depth:None, mx_fts:sqrt min_leaf:8, min_splt:2, n_stim 500 accuracy is:
0.8398629843716549
0:00:11.479930
For the parameters boots:False, mx_depth:None, mx_fts:sqrt min_leaf:8, min_splt:2, n_stim 1000 accuracy is:
0.8370798544208948
0:00:21.666584
For the parameters boots:False, mx_depth:None, mx_fts:sqrt min_leaf:8, min_splt:2, n_stim 1500 accuracy is:
0.8330122029543995
0:00:32.034998
For the parameters boots:False, mx_depth:None, mx_fts:sqrt min_leaf:8, min_splt:5, n_stim 200 accuracy is:
0.8340826375508457
0:00:04.782264
For the